In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical

def load_images_and_labels_from_csv(csv_path, images_dir, image_size=(32, 32), num_classes=43):
    images = []
    labels = []
    data = pd.read_csv(csv_path)
    for idx, row in data.iterrows():
        image_path = os.path.join(images_dir, row['Path'].replace('/', os.sep))
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        if image is not None:
            image = cv2.resize(image, image_size)
            images.append(image)
            labels.append(int(row['ClassId']))
        else:
            print(f"Failed to load image at path: {image_path}")
    images = np.array(images).astype('float32') / 255.0
    labels = to_categorical(np.array(labels), num_classes)
    return images, labels
train_dir = r'C:\Users\Abdullah\Desktop\Derin_Ogrenme\GTSRB'
train_csv = r'C:\Users\Abdullah\Desktop\Derin_Ogrenme\GTSRB\Train.csv'
valid_dir = r'C:\Users\Abdullah\Desktop\Derin_Ogrenme\GTSRB'
valid_csv = r'C:\Users\Abdullah\Desktop\Derin_Ogrenme\GTSRB\Meta.csv'
test_dir = r'C:\Users\Abdullah\Desktop\Derin_Ogrenme\GTSRB'
test_csv = r'C:\Users\Abdullah\Desktop\Derin_Ogrenme\GTSRB\Test.csv'

X_train, y_train = load_images_and_labels_from_csv(train_csv, train_dir)
X_valid, y_valid = load_images_and_labels_from_csv(valid_csv, valid_dir)
X_test, y_test = load_images_and_labels_from_csv(test_csv, test_dir)
print("Training data shape:", X_train.shape, y_train.shape)
print("Validation data shape:", X_valid.shape, y_valid.shape)
print("Test data shape:", X_test.shape, y_test.shape)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# Özelleştirilmiş CNN modelini oluşturalım
def create_custom_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Modeli oluştur
input_shape = (32, 32, 3)  # Örnek olarak 32x32 boyutunda ve 3 kanallı bir giriş
num_classes = 10  # Örnek olarak 10 sınıf
custom_model = create_custom_model(input_shape, num_classes)

# Modeli eğitelim
history_custom = custom_model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_valid, y_valid)
)

# Model performansını değerlendirelim
y_pred = custom_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Performans metrikleri
print("Classification Report:")
print(classification_report(y_true, y_pred_classes))
report = classification_report(y_true, y_pred_classes, output_dict=True)

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Performans özet
test_accuracy = report['accuracy']
precision = report['macro avg']['precision']
recall = report['macro avg']['recall']
f1_score = report['macro avg']['f1-score']
print(f"Accuracy: {test_accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# Transfer öğrenme modelini oluşturalım (MobileNetV2)
input_shape = (128, 128, 3)  # Örnek olarak 128x128 boyutunda ve 3 kanallı bir giriş
num_classes = 10  # Örnek olarak 10 sınıf
base_model_mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
transfer_mobilenet = Sequential([
    base_model_mobilenet,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])
transfer_mobilenet.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitelim (MobileNetV2)
history_mobilenet = transfer_mobilenet.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_valid, y_valid)
)

# Model performansını değerlendirme (MobileNetV2)
y_pred_mobilenet = transfer_mobilenet.predict(X_test)
y_pred_classes_mobilenet = np.argmax(y_pred_mobilenet, axis=1)
y_true_mobilenet = np.argmax(y_test, axis=1)

# Performans metrikleri
print("Classification Report (MobileNetV2):")
print(classification_report(y_true_mobilenet, y_pred_classes_mobilenet))
report_mobilenet = classification_report(y_true_mobilenet, y_pred_classes_mobilenet, output_dict=True)

# Confusion Matrix
cm_mobilenet = confusion_matrix(y_true_mobilenet, y_pred_classes_mobilenet)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_mobilenet, annot=True, fmt="d", cmap='Blues')
plt.title('Confusion Matrix (MobileNetV2)')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Performans özet
test_accuracy_mobilenet = report_mobilenet['accuracy']
precision_mobilenet = report_mobilenet['macro avg']['precision']
recall_mobilenet = report_mobilenet['macro avg']['recall']
f1_score_mobilenet = report_mobilenet['macro avg']['f1-score']
print(f"Accuracy: {test_accuracy_mobilenet:.4f}, Precision: {precision_mobilenet:.4f}, Recall: {recall_mobilenet:.4f}, F1-Score: {f1_score_mobilenet:.4f}")
